# GIVBayesian

Based on the notebook here: https://nbviewer.jupyter.org/github/pycroscopy/papers/blob/master/HTML/SPM/AFM/gIV_sIV_Nat_Comm_2017.html

## Import packages:

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from shutil import copyfile
import tempfile

# I am importing pycroscopy from the original package but you will have to change yours
import sys
sys.path.append('/Users/syz/PycharmProjects/pyUSID/')
sys.path.append('/Users/syz/PycharmProjects/pycroscopy/')

# Change this line to import your custom Dask-based class:
from giv_bayesian_dask import GIVBayesian

/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Make a (temporary) copy of the original file:

In [2]:
orig_path = 'pzt_nanocap_6_just_translation_filt_resh_copy.h5'

with tempfile.TemporaryDirectory() as tmp_dir:
    h5_path = tmp_dir + 'giv.h5'
    copyfile(orig_path, h5_path)

## Open the temporary HDF5 file:

In [3]:
h5_f = h5py.File(h5_path, mode='r+')
h5_grp = h5_f['Measurement_000/Channel_000']

ex_freq = h5_grp.attrs['excitation_frequency_[Hz]']
import pyUSID as usid

h5_resh = h5_f['Measurement_000/Channel_000/Raw_Data-FFT_Filtering_000/Filtered_Data-Reshape_000/Reshaped_Data']

usid.hdf_utils.check_if_main(h5_resh, verbose=True)

True

## Setting up the scientific process here:

Don't modify the scientific parameters

In [4]:
i_cleaner = GIVBayesian(h5_resh, ex_freq, 9, r_extra=110, num_x_steps=250, verbose=True)

Consider calling test() to check results before calling compute() which computes on the entire dataset and writes back to the HDF5 file
ensuring that half steps should be odd, num_x_steps is now 250
Checking for duplicates:


## Calling the compute function here:

In [7]:
h5_bayes_grp = i_cleaner.compute()

Resuming computation. 100% completed already
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Core info {'inproc://10.159.192.90/72297/14': 4}
Scheduler info: {'type': 'Scheduler', 'id': 'Scheduler-daa7934f-16e4-44bf-80f5-59c0ed2ac02f', 'address': 'inproc://10.159.192.90/72297/13', 'services': {'bokeh': 57337}, 'workers': {'inproc://10.159.192.90/72297/14': {'type': 'Worker', 'id': 'inproc://10.159.192.90/72297/14', 'host': '10.159.192.90', 'resources': {}, 'local_directory': '/Users/eju/Downloads/daskUSID/giv_bayesian/dask-worker-space/worker-iqqsmco4', 'name': 'inproc://10.159.192.90/72297/14', 'ncores': 4, 'memory_limit': 4294967296, 'last_seen': 1560882134.207273, 'services': {'bokeh': 57338}, 'metrics': {'cpu': 0.0, 'memory': 50225152

TypeError: can't pickle _thread._local objects

## Preparing to visualize:

In [ ]:
h5_bayes_grp = h5_resh.parent['Reshaped_Data-Bayesian_Inference_000']
bias_interp = np.squeeze(h5_bayes_grp['Spectroscopic_Values'][()])
h5_resistance = h5_bayes_grp['Resistance']
h5_capacitance = h5_bayes_grp['Capacitance']
h5_i_corr_sine = h5_bayes_grp['Corrected_Current']
num_cols = 266

num_rand_pos = 5
chosen_rows = np.random.randint(0, high=5, size=num_rand_pos)
chosen_cols = np.random.randint(0, high=num_cols-1, size=num_rand_pos)

# Visualize results:
Pay attention to the right-most column.
Do you see the red and blue lines with downward and upward facing horns like the ones below? If so, your code is working fine

In [ ]:
from pycroscopy.analysis.utils.giv_utils import plot_bayesian_spot_from_h5
for index in range(num_rand_pos):
    pix_ind = chosen_rows[index] * num_cols + chosen_cols[index]
    fig = plot_bayesian_spot_from_h5(h5_bayes_grp, h5_resh, pix_ind, r_max=300)

In [ ]:
h5_f.close()